First of all, create a new Jupyther notebook, and implement a module that reads files and stores their content in 2 string arrays of file names.

In [3]:
# Import Module
import os
  
# Folder Path
path = "C:/Users/10705/Desktop/Notebooks/labs_ml_naive_bayes/movies_reviews/movies_reviews"
negpath = path +"/neg/"
pospath = path + "/pos"

# Change the directory
os.chdir(negpath)
  
# Read text File
  
  
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        string = f.read()
    return string
    

negstringarray = []
negfilearray = []
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{negpath}\{file}"
  
        # call read text file function
        string = read_text_file(file_path)
        local = [file, string]
        negstringarray.append(string)
        negfilearray.append(file)
os.chdir(pospath)        
posstringarray = []
posfilearray = []
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{pospath}\{file}"
  
        # call read text file function
        string = read_text_file(file_path)
        local = [file, string]
        posstringarray.append(string)
        posfilearray.append(file)
#print(array)
newpath = path + "/sample_new_reviews"
os.chdir(newpath)        
newstringarray = []
newfilearray = []
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{newpath}\{file}"
  
        # call read text file function
        string = read_text_file(file_path)
        local = [file, string]
        newstringarray.append(string)
        newfilearray.append(file)

- Next, you would need to convert the words in each document into a vector of word occurrences. 
You can use the code with stop words from the clustering demo or you can use the `sklearn` module `feature_extraction.text`, 
where you are interested in the `CountVectorizer` (for this one you would need to remove stop words) or in the `TfidfTransformer`. 
The latter assigns a score to each word based on its frequencies across all the documents, 
and thus the words that occur across all the documents (the stop words) get score zero, so there is no need to remove stop words. You can find a nice explanation and an example about tf/idf score [here](https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3). Whatever vectorization technique you chose, you would need to explain it in your own words in a separate markdown cell in your notebook. 

Explaination: The method reads in the strings and build a disctionary after excluding the "build-in" stop words, which are the words that are meaningless. It then converts the string into the frequency array. 

In [4]:
import re
from sklearn.feature_extraction.text import CountVectorizer
#replace everything not in english

vectorizer = CountVectorizer(stop_words="english")
#print(vectorizer.get_stop_words())
allstringarray = negstringarray + posstringarray

for i in range(len(allstringarray)):
    review = allstringarray[i] 
    review = review.encode('ascii', 'ignore').decode()    
    review = review.lower()
    review = re.sub(r"\n", " ", review)
    review = re.sub("^\d+\s|\s\d+\s|\s\d+$", '', review) #remove word+num
    review = re.sub(r'[^\w\s]', '', review) #remove unicode
    review = re.sub("@\S+", " ", review) #remove @
    review = re.sub("https*\S+", " ", review) #remove websites
    review = re.sub("#\S+", " ", review) #remove hashtags
    review = re.sub("\'\w+", '', review) #remove slashes
    review = re.sub('\s{2,}', " ", review) #remove big spaces
    # review = re.sub(r"[0-9]", "",review)#remove numbers
    review = review.strip()
    review = review.replace("_", "")
    allstringarray[i] = review
print(allstringarray[0])
X = vectorizer.fit_transform(allstringarray)

X = X.toarray()
X.shape


plot two teen couples go to a church party drink and then drive they get into an accident one of the guys dies but his girlfriend continues to see him in her life and has nightmares whats the deal watch the movie and sorta find out critique a mindfuck movie for the teen generation that touches on a very cool idea but presents it in a very bad package which is what makes this review an even harder one to write since i generally applaud films which attempt to break the mold mess with your head and such lost highway memento but there are good and bad ways of making all types of films and these folks just didnt snag this one correctly they seem to have taken this pretty neat concept but executed it terribly so what are the problems with the movie well its main problem is that its simply too jumbled it starts off normal but then downshifts into this fantasy world in which you as an audience member have no idea whats going on there are dreams there are characters coming back from the dead th

(2005, 48103)

Once you have a vector for each review, you can add the labels *pos* or *neg*, depending on the directory (as we did in cat/dog classification demo), and then divide the dataset into training and testing.

In [5]:
import numpy as np
Y_orig = np.ones((1000,)) # 1 - 1000 are neg reviews so our label is 1
Y_orig = np.concatenate((Y_orig, np.zeros((1005,)))) # 1000 - 2005 are pos reviews so our label is 0
Y = Y_orig.reshape(-1)
print("At position 3 should be a neg:", Y[3])
print("At position 2004 should be a pos:", Y[2004])
Y.shape
print(Y)

At position 3 should be a neg: 1.0
At position 2004 should be a pos: 0.0
[1. 1. 1. ... 0. 0. 0.]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)


Now use the train dataset to build a Naive Bayes model. You can use the `sklearn` module `naive_bayes` from [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.naive_bayes) to accomplish this task. Carefully select the correct classifier for the data at hand by reading about different classification options.

In [11]:
from sklearn.naive_bayes import MultinomialNB
mclf = MultinomialNB()
print(X_train.shape, Y_train.shape)
mclf.fit(X_train, Y_train)
train_score = mclf.score(X_train, Y_train)
print(train_score)
#test_score = mclf.score(X_test,Y_test)
#print(test_score)

#get test score
from sklearn import metrics
prediction_test2 = mclf.predict(X_test)
score2 = metrics.accuracy_score(Y_test, prediction_test2)
print('Total accuracy classification score with MultinomialNB: {}'.format(score2))

(1343, 48103) (1343,)
0.9865971705137752
Total accuracy classification score with MultinomialNB: 0.8338368580060423


Finally, find 5 new movie reviews on the internet which include a numeric or star rating (known to be positive or negative), and try to classify them into positive/negative using your classifier. Report and discuss the results in a separate markdown cell.

In [8]:
newpath = path + "/sample_new_reviews"
os.chdir(newpath)        
newstringarray = []
newfilearray = []
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{newpath}\{file}"
  
        # call read text file function
        string = read_text_file(file_path)
        local = [file, string]
        newstringarray.append(string)
        newfilearray.append(file)
for i in range(len(newstringarray)):
    review = review.encode('ascii', 'ignore').decode()
    review = newstringarray[i] 
    review = review.lower()
    review = re.sub(r"\n", " ", review)
    review = re.sub("^\d+\s|\s\d+\s|\s\d+$", '', review)
    review = re.sub(r'[^\w\s]', '', review)
    review = re.sub("@\S+", " ", review) #remoove @
    review = re.sub("https*\S+", " ", review)
    review = re.sub("#\S+", " ", review)
    review = re.sub("\'\w+", '', review)
    review = re.sub('\s{2,}', " ", review)     
    review = review.strip()
    review = review.replace("_", "")
    newstringarray[i] = review


In [9]:
sample = vectorizer.transform(newstringarray)
vectorizer.get_feature_names_out()
sample.shape

(5, 48103)

In [10]:


print(mclf.predict_proba(sample.toarray()))

[[0.99735668 0.00264332]
 [0.91156945 0.08843055]
 [0.72074823 0.27925177]
 [0.81346709 0.18653291]
 [0.00366145 0.99633855]]


The result is that for the first one it successfully recognizes a positive review, and the 5th one recognizes the negative review. The second, third and forth ones, although they recognizes the result correctly, the probability of wrong classification is still around or higher than 10% since those reviews uses negative words such as "bored", "unfilled" so it is distracting to the classification. However, the remaining part outweights the negative words and still result in a positive result